# Explain Dataset

#### Acc_Intakes.csv 
- Age Upon Intake: The age of the animal when it was brought to the facility
- Animal ID: A unique identifier for each animal.
- Animal Type: The species of the animal. In both examples, this is 'Dog'.
- Breed: The breed of the animal.
- Color: The color of the animal's coat.
- Datetime: The date and time when the animal was intake.
- Datetime2: This seems to be a repeat of the intake date and time
- Found Location: The location where the animal was found before being brought to the facility.
- Intake Condition: The health or physical condition of the animal at intake.
- Intake Type: The circumstances under which the animal was brought to the facility.
- Name: The name given to the animal. 
- Sex Upon Intake: The gender and reproductive status of the animal when it arrived.

#### Acc_Outcomes.csv
- Age Upon Outcome: This is the age of the animal at the time they left the facility
- Animal ID: A unique identifier assigned to each animal.
- Animal Type: The species of the animal.
- Breed: The breed of the animal.
- Color: The color of the animal's coat.
- Date of Birth: The birth date of the animal. 
- Datetime: The date and time when the animal left the facility.
- Monthyear: This seems to be a repeat of the outcome date and time
- Name: The name of the animal
- Outcome Subtype: Additional details about the outcome.
- Outcome Type: The general category of the outcome.
- Sex Upon Outcome: The gender and reproductive status of the animal when it left the facility. 

# Clearn data before analyze.
#### Import pandas and numpy library

In [4]:
import pandas as pd
import numpy as np

#### Read file aac_intakes.csv and save as 'aac_intakes' dataframe

In [13]:
input_data=pd.read_csv('C:\\Users\\lemin\\Downloads\\Day_5 (1)\\Demo\\aac_intakes.csv')
output_data=pd.read_csv('C:\\Users\\lemin\\Downloads\Day_5 (1)\\Demo\\aac_outcomes.csv')

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
C:\Users\lemin\AppData\Local\Temp\ipykernel_13352\2692003013.py:2: SyntaxWarning: invalid escape sequence '\D'
  output_data=pd.read_csv('C:\\Users\\lemin\\Downloads\Day_5 (1)\\Demo\\aac_outcomes.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\lemin\\Downloads\\Day_5 (1)\\Demo\\aac_intakes.csv'

Let's have a look at dataframe and change data type as neededneeded

In [ ]:
input_data.info()

In [ ]:
#change data type of input_data
input_data[['datetime','datetime2']]=input_data[['datetime','datetime2']].apply(pd.to_datetime)
input_data.info()
input_data.sort_values(by='datetime')

In [ ]:
output_data.info()
output_data[['date_of_birth','datetime','monthyear']]=output_data[['date_of_birth','datetime','monthyear']].apply(pd.to_datetime)


In [ ]:
output_data.info()

Duplicated rows can be removed before analysis.

In [ ]:
#check duplicates, and see how data is duplicated
input_data_dup=input_data[input_data.duplicated(keep=False)]
input_data_dup.sort_values(by='animal_id').head()

In [ ]:
# remove duplicate
input_data=input_data.drop_duplicates()
input_data.sort_values(by='animal_id').head(10)
input_data.shape
input_data=input_data.dropna(subset=['animal_id','datetime','datetime2'])
input_data.shape

In [ ]:
input_data_up = input_data .sort_values(by='datetime', ascending=False)  # Sort by date (latest first)
input_data_up = input_data.drop_duplicates(subset='animal_id', keep='first')  # Keep the first (latest)
input_data_up.shape

Similar to acc_outcome, we will remove duplicate entry uupdate the latest time of the animal intakes with the latest time

In [ ]:
output_data=output_data.drop_duplicates()
output_data=output_data.dropna(subset=['animal_id'])
output_data=output_data.sort_values(by='datetime', ascending=False)  # Sort by date (latest first)
output_data= output_data.drop_duplicates(subset='animal_id', keep='first')  # Keep the first (latest)
output_data.shape

# Answers question

#### Question number 1: What is the distribution of the types of animals in the shelter?

In [ ]:
output_data.head()

#### Question number 2: Is there an area where more pets are found?. Find the top 5 location where pet are found

In [ ]:
# cach1: get the first afer sorting valuesvalues
count_location=input_data.groupby('found_location')['animal_id'].count().sort_values(ascending=False).head(5)
count_location

In [ ]:
# cach 2: can get the value with the same result at top 5
location_group=input_data['found_location'].value_counts()
threshold=location_group.iloc[4]
top_5=location_group[location_group>=threshold]
top_5

#### Question number 3: What is the average number of pets found in a month in the year 2015?. Are there months where there is a higher number of animals found?

In [ ]:
# Filter data for 2015
data_2015 = input_data[(input_data['datetime'] >= "2015-01-01") & (input_data['datetime'] <= "2015-12-31")]

# Calculate the average number of found animals per month in 2015
avg_2015 = data_2015.groupby(data_2015['datetime'].dt.month)['animal_id'].count().mean()
print(f"Average number of animals found per month in 2015: {avg_2015}")

# Group by month and count occurrences of 'animal_id'
animal_2015 = data_2015.groupby(data_2015['datetime'].dt.month)['animal_id'].count().sort_values()

# Get the highest number of found animals in a single month (top 1)
top_1 = animal_2015.iloc[0]  # Highest number of found pets in a month
month_1 = animal_2015[animal_2015 == top_1]  # Get the months with the top count
months= ','.join(map(str, month_1.index.tolist()))
# Convert months to a string without square brackets
months = ', '.join(map(str, month_1.index.tolist()))

# Print the month(s) and the number of found pets
print(f"The month(s) with the highest number of found pets in 2015 is/are: {months} with {top_1} found pets.")



#### Question number 4: What is the ratio of incoming pets vs. adopted pets

In [ ]:
# check the number of animals incoming
in_animal=input_data_up.shape[0]
# Number of adopted pets
out_animal=output_data[output_data['outcome_type']=='Adoption'].shape[0]
print ('Ratio of incoming pets and adopted pets', round(100*out_animal/in_animal,3),'%')

#### Question number 5: What are the adoption rates for specific breeds?
#### Find the top 5 dog breeds in the shelter (based on count) and then find the adoption percentage of each breed.

In [ ]:
# total intakes of dog breedsbreeds
dog_data=input_data_up[input_data_up['animal_type']=='Dog']
breed_count=dog_data['breed'].value_counts()
threshold=breed_count.iloc[4]
top_5=breed_count[breed_count>=threshold]
top_5_df = top_5.reset_index()  # Converts the Series to DataFrame
top_5_df.columns = ['breed', 'intakes']  # Rename the columns

top_5_intakes=top_5.index.to_list()
## Find the adoption for dog breedsbreeds
dog_outcome = output_data[(output_data['animal_type'] == 'Dog') & (output_data['outcome_type'] == 'Adoption')]

dict_dog = {}

# Assuming top_5_intakes is a list of the top 5 breed values
for i in top_5_intakes:
    count_breed = dog_outcome[dog_outcome['breed'] == i].shape[0]  # Count occurrences of breed
    dict_dog[i] = count_breed  # Add breed and its count to the dictionary

# Convert dictionary to DataFrame for better readability
dog_df = pd.DataFrame(list(dict_dog.items()), columns=['breed', 'adoption count'])
merged_df = pd.merge(dog_df, top_5_df, on='breed', how='inner')
merged_df['Adoption Rate']=merged_df['adoption count']*100/merged_df['intakes'] 
merged_df['Adoption Rate'] = round(merged_df['Adoption Rate'],2).astype(str) + ' %'
print(merged_df)

#### Question number 6: About how many animals are spayed/neutered each month?
#### This will help the shelter allocate resources and staff. Assume that all intact males and females will be spayed/neutered.


In [ ]:
#Slice data to sex_intakess, sex_outcomes

sex_intakes=input_data_up[['animal_id','sex_upon_intake']]
sex_outcomes_new=output_data[['animal_id','sex_upon_outcome','monthyear']]
print(sex_intakes.shape[0])
print(sex_outcomes_new.shape[0])

In [ ]:
#Find the animals are spayed or neutered by staff
#Step 1:Find the animals that haven't been spayed before
#Step 2: Find the animals that have been spay after outcomes
#Step 3: Filter data based on step1, step 2
sex_intakes['sex_upon_intake'].unique()

In [ ]:
#Step 1:Find the animals that haven't been spayed before
intake_no_spay = sex_intakes[
    (sex_intakes['sex_upon_intake'] != 'Spayed Female') & 
    (sex_intakes['sex_upon_intake'] != 'Neutered Male')
]


In [ ]:
#Step 2: Find the animals that have been spay after outcomes
outcome_spay=sex_outcomes_new[
        (sex_outcomes_new['sex_upon_outcome'] == 'Spayed Female') |
    (sex_outcomes_new['sex_upon_outcome'] == 'Neutered Male')
]


In [ ]:
intersection_data=pd.merge(intake_no_spay,outcome_spay,how ='inner', on =['animal_id'])
intersection_data['month_year'] = intersection_data['monthyear'].dt.strftime('%Y-%m')
month_number=intersection_data['month_year'].nunique()
print('Average animals that are spayed or neuteredutered each month', round(intersection_data.shape[0]/month_number))

#### Question number 7
How many animals in the shelter are repeats?, Which animal was returned to the shelter the most?
This means the animal has been brought in more than once.

In [ ]:
ids_in = input_data.animal_id
ids_out = output_data.animal_id

Duprows_aac_intakes = input_data[ids_in.isin(ids_in[ids_in.duplicated()])].sort_values("animal_id")
Duprows_aac_outcomes = output_data[ids_out.isin(ids_out[ids_out.duplicated()])].sort_values("animal_id")
Duprows_aac_intakes.shape[0]

In [ ]:
# In aac_intakes, we find the most animal_type was returned to the shelter
id_in_dup = input_data[input_data['animal_id'].duplicated()]['animal_id']
dup_id=input_data[input_data['animal_id'].isin(id_in_dup)]
print('There are', dup_id.shape[0] ,'animals in the shelter that are repeats')

In [ ]:
top_repeat=dup_id['animal_type'].value_counts()
print(top_repeat.index[0], 'having the highest times of return', top_repeat.iloc[0])

#### Question number 8
What are the adoption rates for different colorings?
Find the top 5 colorings in the shelter (based on count) and then find the adoption percentage of each color.


In [ ]:
# Rewrite in different solution
# Find rows have 'Dog' breed in shelter
dog_data=input_data_up[input_data_up['animal_type']=='Dog']
# Find the top 5 'Dog' color in shelter
top_5_color=dog_data.groupby('color')['animal_id'].count().sort_values(ascending=False).head(5)
# Create a list of values from top_5_colorcolor other way:
list_values_top5=[]
for i in range(0, top_5_color.shape[0]):
    value_top=top_5_color.iloc[i]
    list_values_top5.append(value_top)


# List values find the number of top 5 dogs in shelter had record in breedDog_outcomes
#Step 1: filter data by type='Dog' and 'Adoption'
dog_adoption=output_data[(output_data['animal_type']=='Dog')& (output_data['outcome_type']=='Adoption')]
#Step 2: Group by color:
dog_adoption_color=dog_adoption['color'].value_counts()
#Step 3: Loop over index but index of color of adoption is the same color as top5
list_adoption_top5=[]
for values in top_5_color.index:
    data_row = dog_adoption_color[dog_adoption_color.index == values]
    value_row=data_row.loc[values]
    list_adoption_top5.append(value_row)
print(list_adoption_top5)
print(list_values_top5)
top_5_list=top_5_color.index.tolist()
# Link all lists together:
top5_color_df=pd.DataFrame(list(zip(top_5_list,list_values_top5,list_adoption_top5)), columns=['color','total animals','adoption number'])
top5_color_df['adoption rate']=round(top5_color_df['adoption number']*100/top5_color_df['total animals'],2).astype(str)+'%'
top5_color_df

In [ ]:

top_5_color_list=top_5_color.index
top_5_adoption=dog_adoption_color[dog_adoption_color.index.isin(top_5_color_list)]

# Step 1: Convert Series to DataFrames
df_top_5_color = pd.DataFrame(top_5_color)  # This will create a DataFrame with one column
df_top_5_color.columns = ['total_animals']  # Renaming the column for clarity
df_top_5_color.reset_index(inplace=True) # Add color as a new column

# Step 2: Data Frame of number of adoptions
df_top_5_adoption = pd.DataFrame(top_5_adoption)  # This will create a DataFrame with one column
df_top_5_adoption.columns = ['total_adoption']  # Renaming the column for clarity
df_top_5_adoption.reset_index(inplace=True) 
# Step 3: Merge the DataFrames on 'color'
merged_data=pd.merge(df_top_5_adoption,df_top_5_color, how='inner', on='color')
# Perform the division to calculate the adoption rate
merged_data['adoption_rate'] = round((merged_data['total_adoption'] * 100 / merged_data['total_animals']),2)

# Add '%' symbol to the adoption rate
merged_data['adoption_rate'] = merged_data['adoption_rate'].astype(str) + '%'

# Display the updated merged data
print(merged_data)

#### Question number 9
What are the adoption rates for the following age groups?
- baby: 4 months and less
- young: 5 months - 2 years
- adult: 3 years - 10 years
- senior: 11+

In [ ]:
adoption_age_group=output_data[output_data['outcome_type']=='Adoption']
#ignore entries having na values
adoption_age_group=adoption_age_group[adoption_age_group['age_upon_outcome'].notna()]
#Find data have month
adoption_month=adoption_age_group[adoption_age_group['age_upon_outcome'].str.contains('month')]
#Find data have year
adoption_year=adoption_age_group[adoption_age_group['age_upon_outcome'].str.contains('year')]

Lower_1_month =adoption_age_group[adoption_age_group['age_upon_outcome'].str.contains('week|day')]

#### Filter the data between baby, young, aldult and senior animal by index number.

list_Baby_index = []
list_Young_index = []
list_Aldult_index = []
list_Senior_index = []

##Find data in age_upon_months datafame
# Between 1 month and 1 year
list_Baby_index = []  # Initialize the list

for i in range(adoption_month.shape[0]):  # Loop through all rows
    value_i_list = adoption_month['age_upon_outcome'].iloc[i].split(' ')  # Split the string by space
    extracted_month = int(value_i_list[0])  # Convert to integer
    
    if extracted_month < 5:  # Check if the extracted month is less than 5
        list_Baby_index.append(adoption_month.age_upon_outcome.index[i])  # Append the index i to the list

    else: 
         list_Young_index.append(adoption_month.age_upon_outcome.index[i]) 
for x in range(adoption_year.shape[0]):  # Loop through all rows
    value_x_list = adoption_year['age_upon_outcome'].iloc[x].split(' ')  # Split the string by space
    extracted_year = int(value_x_list[0])  # Convert to integer
    
    if extracted_year <=2:  
        list_Young_index.append(adoption_month.age_upon_outcome.index[x])

    elif extracted_year<=10: 
         list_Aldult_index .append(adoption_month.age_upon_outcome.index[x]) 
    else: list_Senior_index.append(adoption_month.age_upon_outcome.index[x]) 
    
print('Baby rate: ',round(100 / adoption_age_group.shape[0] * int(len(list_Baby_index) + Lower_1_month.shape[0]),3), '%')
print('Young rate: ',round(100 / adoption_age_group.shape[0] * int(len(list_Young_index)),3), '%')
print('Aldult rate: ',round(100 / adoption_age_group.shape[0] * int(len(list_Aldult_index)),3), '%')
print('Senior rate: ',round(100 / adoption_age_group.shape[0] * int(len(list_Senior_index)),3), '%')   



#### Question number 10:
If spay/neuter for a dog costs $100 and a spay/neuter for a cat costs $50,
how much did the shelter spend in 2015 on these procedures?

In [ ]:
intersection_data_2015= intersection_data[(intersection_data['monthyear'] > "2015-01-01") & (intersection_data['monthyear']< "2015-12-31")]
intersection_data_2015_type=pd.merge(intersection_data_2015, output_data, how='inner', on='animal_id')
intersection_data_2015_type.drop_duplicates()

a=intersection_data_2015_type.groupby('animal_type')['animal_id'].count()
# Filter data for 2015, 6072
intersection_data_2015 = intersection_data[(intersection_data['monthyear'] > "2015-01-01") & (intersection_data['monthyear'] < "2015-12-31")]

# Merge with output_data on 'animal_id'
intersection_data_2015_type = pd.merge(intersection_data_2015, output_data, how='inner', on='animal_id')

# Group by 'animal_type' and count 'animal_id'
a = intersection_data_2015_type.groupby('animal_type')['animal_id'].count()

total_sum = 0
for x,y in a.items():
    if x == 'Cat':
        total_sum += y * 50  # Add 50 for each 'Cat'
    elif x == 'Dog':
        total_sum += y * 100  # Add 100 for each 'Dog'

print("Total Sum:", total_sum)